<a href="https://colab.research.google.com/github/cdiegor/Metaheuristicas/blob/main/MoMu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problema da Mochila Multimodal

## Introdução

## Problema em dois níveis

In [ ]:
class MoMu:
  def __init__(self, n:int, m:int):
    self.n = n            # Número de objetos
    self.m = m            # Quantidade de modos por objeto
    self.valor = [][]     # Valores de cada objeto em cada modo
    self.peso = [][]      # Pesos de cada objeto em cada modo
    self.capacidade       # Capacidade máxima da mochila

    self.modo = [0]*n     # Representação da solução
                          # Modo 0 => Peso = 0 e Valor = 0
    #self.valorObj = 0

  def valorObj(self):     # Valor da função objetivo para os modos escolhidos
    val = 0
    res = capacidade
    for j in range(self.n):
      val = val + self.valor[j][ self.modo[j] ]
      res = res - self.peso[j][ self.modo[j] ]
    if res < 0: val = 0
    return val

  def residual(self):
    res = capacidade
    for j in range(self.n):
      res = res - self.peso[j][ self.modo[j] ]
    return res



## Mochila 0-1 por Colônia de Formigas

In [1]:
import random
import copy
import numpy as np

class ACO:

  def __init__(self, MM:MoMu, num_formigas:int, num_iter: int):
    self.num_formigas = num_formigas
    self.num_iter = num_iter
    self.pop:MoMu = []*MM
    self.feromonio:float = []
    self.decaimento:float = 0.9
    self.MM = MM


  def gerar_populacao(self):
    for i in range(self.num_formigas):
      ordem = np.argsort(-self.feromonio)
      cap = self.pop[i].capacidade
      for j in range(self.MM.n):
        indice = min( random.geometric(p=0.5), len(ordem)-1 )
        escolhido = ordem[indice]
        if self.pop[i].peso[escolhido][self.MM.modo[escolhido]] <= cap:
          cap = cap - self.pop[i].peso[escolhido][self.MM.modos[escolhido]]
        else:
          self.pop[i].modo[escolhido] = 0
        ordem.remove(escolhido)

  def iniciar_feromonios(self):
    for j in range(self.MM.n):
      self.feromonio[j] = self.MM.valor[j][self.MM.modo[j]] / self.MM.peso[j][self.MM.modo[j]]

  def espalhar_feromonio(self, melhor:MoMu):
    # Decaimento
    self.feromonio = self.feromonio * self.decaimento

    # Reforço pela melhor solução
    for j in range(self.MM.n):
      if melhor.modo[j] > 0:
        self.feromonio[j] = self.feromonio[j] + self.MM.valor[j]/self.MM.peso[j]


  def rodar(self):
    melhor_de_todas = self.MM
    self.iniciar_feromonios()
    for iter in range(self.num_iter):
      self.gerar_populacao()
      melhor = max(self.pop, key = lambda x : x.valorObj())
      if melhor.valorObj() > melhor_de_todas.valorObj():
        melhor_de_todas = melhor

      self.espalhar_feromonio(melhor)

    return melhor


SyntaxError: ignored

## Mochila Multimodal por Religação de Caminhos

In [ ]:
def busca_local(S: MoMu, k:int):
  melhor = copy.deepcopy(S)
  cap = melhor.residual()
  val = melhor.valorObj()

  for j in range(S.n):
    if j != k:
      for m in range(S.m):
        if m != S.modos[j]:
          if cap - S.pesos[j][S.modos[j]] + S.pesos[j][m] > 0 and
             val - S.valores[j][S.modos[j]] + S.valores[j][m] > melhor.valorObj
             melhor = copy.deepcopy(S)
             melhor.modos[j] = m
  return melhor

In [ ]:
def religacao(S:MoMu, R:MoMu):
  T = copy.deepcopy(S)
  melhor = copy.deepcopy(max([S,R], key = lambda x : x.valorObj))
  for j in range(S.MM.n):
    if S.modo[j] != R.modo[j]:
      T.modo[j] = R.modo[j]
      T = busca_local(T, j)
      if T.valorObj() > melhor.valorObj():
        melhor = copy.deepcopy(T)
  return melhor